<a href="https://colab.research.google.com/github/daryoush/minGPT/blob/master/startjulia1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/My Drive/
#! git clone https://github.com/daryoush/minGPT.git
%cd minGPT
! git pull
#! pip install <desired-python-library>

/content/gdrive/My Drive
/content/gdrive/My Drive/minGPT
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/daryoush/minGPT
   22fab47..34ca2ed  master     -> origin/master
Updating 22fab47..34ca2ed
Fast-forward
 startjulia1_5.ipynb | 577 ++++++++++++++++++++++++----------------------------
 1 file changed, 261 insertions(+), 316 deletions(-)


# New Section

In [3]:
### NOTE THIS WORKS AS PYTHON, IT ONLY WORKS WHEN THE RUNTIME COMPLAINS ABOUT JULIA RUNTIME, SO IT REVERTS TO PYTHON. AFTER JULIA IS INSTALLED, CHANGE TO JULIA RUNTIME

!curl -sSL "https://julialang-s3.julialang.org/bin/linux/x64/1.5/julia-1.5.0-linux-x86_64.tar.gz" -o julia.tar.gz
!tar -xzf julia.tar.gz -C /usr --strip-components 1
!rm -rf julia.tar.gz*
!julia -e 'using Pkg; pkg"add IJulia; precompile"'
!julia -e 'using Pkg; Pkg.add([ "CUDA", "Flux","Zygote","Profile", "TensorBoardLogger", "Logging",\
    "Random","InformationMeasures", "DataStructures", "Transformers","Statistics", "Dates",\
    "BSON",\
    "Images",\
    "MLDatasets",\
    "Parameters",\
    "Printf",\
    ]);' ###Pkg.precompile()'
!echo "DONE"

 Installing known registries into `~/.julia`
######################################################################## 100.0%
      Added registry `General` to `~/.julia/registries/General`
  Resolving package versions...
  Installed ZeroMQ_jll ────── v4.3.2+5
  Installed Conda ─────────── v1.5.0
  Installed SoftGlobalScope ─ v1.1.0
  Installed ZMQ ───────────── v1.2.1
  Installed JSON ──────────── v0.21.1
  Installed VersionParsing ── v1.2.0
  Installed JLLWrappers ───── v1.1.3
  Installed IJulia ────────── v1.23.1
  Installed Artifacts ─────── v1.3.0
  Installed MbedTLS ───────── v1.0.3
  Installed MbedTLS_jll ───── v2.16.8+1
  Installed Parsers ───────── v1.0.15
######################################################################## 100.0%
######################################################################## 100.0%
Updating `~/.julia/environments/v1.5/Project.toml`
  [7073ff75] + IJulia v1.23.1
Updating `~/.julia/environments/v1.5/Manifest.toml`
  [56f22d72] + Artifacts v1.3.0
  

**NOW GO TO EDIT NOTEBOOK SETTING AND SET IT TO JULIA**
For adding new packages, it should be in python or the syntax may need to be in Julia!

In [ ]:
[1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [ ]:
import Pkg; Pkg.add("BenchmarkTools")

   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
  Installed BenchmarkTools ─ v0.5.0
Updating `~/.julia/environments/v1.5/Project.toml`
  [6e4b80f9] + BenchmarkTools v0.5.0
Updating `~/.julia/environments/v1.5/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v0.5.0


In [ ]:
using BenchmarkTools

M = rand(2048, 2048)
@benchmark M^2

BenchmarkTools.Trial: 
  memory estimate:  32.00 MiB
  allocs estimate:  2
  --------------
  minimum time:     417.434 ms (0.00% GC)
  median time:      431.325 ms (0.00% GC)
  mean time:        435.868 ms (0.73% GC)
  maximum time:     471.040 ms (0.71% GC)
  --------------
  samples:          12
  evals/sample:     1

In [ ]:
if ENV["COLAB_GPU"] == "1"
    using CUDA

    M_gpu = cu(M)
    @benchmark CUDA.@sync M_gpu^2
else
    println("No GPU found.")
end

BenchmarkTools.Trial: 
  memory estimate:  416 bytes
  allocs estimate:  11
  --------------
  minimum time:     2.757 ms (0.00% GC)
  median time:      4.951 ms (0.00% GC)
  mean time:        5.025 ms (0.03% GC)
  maximum time:     9.150 ms (17.12% GC)
  --------------
  samples:          994
  evals/sample:     1

In [ ]:
M_gpu = nothing
M=nothing

In [1]:
cd("/content/gdrive/MyDrive/minGPT")
pwd()

"/content/gdrive/MyDrive/minGPT"

In [2]:
using Flux: onehot
	using Transformers: Vocabulary, decode
	using Random
	using Flux: throttle, @epochs, ADAMW, ADAM, RADAM, Momentum, Optimiser, logitcrossentropy, params
	using Flux: softmax, onecold, WeightDecay
	using DataStructures: Queue, enqueue!, dequeue!
	Random.seed!(42)


MersenneTwister(UInt32[0x0000002a], Random.DSFMT.DSFMT_state(Int32[964434469, 1073036706, 1860149520, 1073503458, 1687169063, 1073083486, -399267803, 1072983952, -909620556, 1072836235  …  -293054293, 1073002412, -1300127419, 1073642642, 1917177374, -666058738, -337596527, 1830741494, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x0000000000

In [3]:
import Pkg; Pkg.add("NNlib")
Pkg.add("Distributions")
Pkg.add("Juno")

   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [4]:
using Flux

module Datasets include("onJulia/src/datasets.jl") end

Main.Datasets

In [5]:
f="input.txt"
s = open(f) do file
      read(file, String)
end
data = s 
xxx=10    # I don't want the out put of the data assignment to be printed

10

In [6]:
data[100:150]

"u are all resolved rather to die than to famish?\n\nA"

In [7]:
using CUDA
CUDA.memory_status()

Effective GPU memory usage: 0.73% (109.875 MiB/14.726 GiB)
CUDA allocator usage: 0 bytes
binned usage: 0 bytes (0 bytes allocated, 0 bytes cached)


In [8]:

dataSetlength, vocab, sample = Datasets.asDataset(data)
batch(dataLength,batchLength ) = hcat((sample(dataLength) for x in 1:batchLength)...)

datasize = length(s) = 1115394


batch (generic function with 1 method)

In [9]:
## ---
module Model include("onJulia/src/model.jl") end

lengthOfSentences=128

128

In [10]:
model=Model.minGPT(numberAttnLayers=4, maxBlockSize=lengthOfSentences, vocabSize=length(vocab)) |> gpu

Main.Model.minGPT{Transformers.Basic.Embed{Float32,CuArray{Float32,2}},Main.Model.GPTPositionEmbedding{Float32,CuArray{Float32,2}},Chain{NTuple{4,Transformers.Basic.Transformer{Transformers.Basic.MultiheadAttention{Dense{typeof(identity),CuArray{Float32,2},CuArray{Float32,1}},Dense{typeof(identity),CuArray{Float32,2},CuArray{Float32,1}},Dense{typeof(identity),CuArray{Float32,2},CuArray{Float32,1}},Dense{typeof(identity),CuArray{Float32,2},CuArray{Float32,1}},Dropout{Float64,Colon}},LayerNorm{CuArray{Float32,1}},Transformers.Basic.PwFFN{Dense{typeof(gelu),CuArray{Float32,2},CuArray{Float32,1}},Dense{typeof(identity),CuArray{Float32,2},CuArray{Float32,1}}},LayerNorm{CuArray{Float32,1}},Dropout{Float64,Colon}}}},Dropout{Float64,Colon},LayerNorm{CuArray{Float32,1}},Transformers.Basic.Positionwise{Tuple{Dense{typeof(identity),CuArray{Float32,2},CuArray{Float32,1}}}}}(Embed(512), Main.Model.GPTPositionEmbedding{Float32,CuArray{Float32,2}}(Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0

In [14]:
@show modelParmaLength=sum(length(p) for p in params(model))
CUDA.memory_status()

modelParmaLength = sum((length(p) for p = params(model))) = 12743746
Effective GPU memory usage: 1.06% (159.875 MiB/14.726 GiB)
CUDA allocator usage: 48.856 MiB
binned usage: 48.856 MiB (48.856 MiB allocated, 0 bytes cached)


In [12]:
# for a batch of sentences, extract 1st to one before last character as input
# and 2nd to last char of each sentence as prediction.  Shape the predictred
# values as a long one dim series of values, one hot encode them using the vocab
numberOfSentencesInBatch=512

trainingDataPair(b)=(b[1:end-1, :], onehot(vocab, reshape(b[2:end,:], :)))
trainingData(eachSampleSize=lengthOfSentences,batchSize=numberOfSentencesInBatch)= [trainingDataPair(batch(eachSampleSize,batchSize))
		for i in 1:dataSetlength/batchSize]  # count each sentence, no matter its length as one sample

function loss(x,y)
	yhat=model(x)
	logitcrossentropy(reshape(yhat,size(yhat)[1], :), y)
end


function validate()
	s=collect("Persia")
	for i in 1:20
		out=model(vocab(s))
		push!(s, decode(vocab,onecold(softmax(out)[:,end])))
	end
	join(s)
end

validate (generic function with 1 method)

In [13]:
#opt = RADAM()
opt = ADAM( 0.0006, (0.9, 0.95))

## ---
module Train include("onJulia/src/train.jl") end

Main.Train

In [54]:
function sampleFromModel(s="Whe", minprob=.05) 
	function explore(x, minprob)
		x[1] < minprob && return currentRes
		out=model(vocab(collect(x[2])))
		choices=sort(collect(zip(softmax(out[:,end]), decode(vocab, collect(1:size(out)[1])))), rev=true)
		options = [(x[1]*np, x[2]*nc) for (np,nc) in choices if (x[1]*np) > minprob ]
	end
  minprob = minprob |> gpu
	toexplore=Queue{Tuple{Float32,String}}() |> gpu
	enqueue!(toexplore,(1, s))  #start with the initial string and prob 1
	results=[] |> gpu

  
	while !isempty(toexplore)
		nxt=dequeue!(toexplore) |> gpu
		res=explore(nxt, minprob) |> gpu
		if isempty(res)  # go as long as you can't add any more character and satisfy the prob
			push!(results, nxt)
			continue #continue emptying the queue
		end
		for r in res
			enqueue!(toexplore, r)  ## add elements for further exploration
		end
	end
    #elements of the results are (prob, string), sort from high to low of the 
	#longest match with highest prob
	sort(results, by=x->(length(x[2]), x[1]), rev=true)
end


sampleFromModel (generic function with 3 methods)

In [56]:

@show sampleFromModel("Oh lord!", 0.01)


sampleFromModel("Oh lord!", 0.01) = Any[(0.04966428f0, "Oh lord!&"), (0.041296836f0, "Oh lord!."), (0.036131218f0, "Oh lord!;"), (0.032799706f0, "Oh lord!i"), (0.026509747f0, "Oh lord!P"), (0.024511797f0, "Oh lord!D"), (0.023699107f0, "Oh lord!,"), (0.022793079f0, "Oh lord!w"), (0.02246859f0, "Oh lord!I"), (0.021919023f0, "Oh lord!z"), (0.021403074f0, "Oh lord! "), (0.021033933f0, "Oh lord!X"), (0.02083366f0, "Oh lord!d"), (0.019290581f0, "Oh lord!M"), (0.018754318f0, "Oh lord!x"), (0.018630486f0, "Oh lord!C"), (0.017449822f0, "Oh lord!O"), (0.017154882f0, "Oh lord!A"), (0.016983192f0, "Oh lord!u"), (0.016346578f0, "Oh lord!K"), (0.016321437f0, "Oh lord!Z"), (0.015822938f0, "Oh lord!3"), (0.015720326f0, "Oh lord!\n"), (0.014881906f0, "Oh lord!W"), (0.014859531f0, "Oh lord!k"), (0.014652494f0, "Oh lord!L"), (0.013755309f0, "Oh lord!p"), (0.013582438f0, "Oh lord!:"), (0.013536628f0, "Oh lord!b"), (0.013491166f0, "Oh lord!g"), (0.013217933f0, "Oh lord!F"), (0.013122111f0, "Oh lord!S"), (0

50-element Array{Any,1}:
 (0.04966428f0, "Oh lord!&")
 (0.041296836f0, "Oh lord!.")
 (0.036131218f0, "Oh lord!;")
 (0.032799706f0, "Oh lord!i")
 (0.026509747f0, "Oh lord!P")
 (0.024511797f0, "Oh lord!D")
 (0.023699107f0, "Oh lord!,")
 (0.022793079f0, "Oh lord!w")
 (0.02246859f0, "Oh lord!I")
 (0.021919023f0, "Oh lord!z")
 (0.021403074f0, "Oh lord! ")
 (0.021033933f0, "Oh lord!X")
 (0.02083366f0, "Oh lord!d")
 ⋮
 (0.012420357f0, "Oh lord!f")
 (0.012365066f0, "Oh lord!H")
 (0.012332071f0, "Oh lord!\$")
 (0.012264194f0, "Oh lord!T")
 (0.011589125f0, "Oh lord!c")
 (0.011587756f0, "Oh lord!j")
 (0.011492992f0, "Oh lord!t")
 (0.010930371f0, "Oh lord!y")
 (0.010331361f0, "Oh lord!e")
 (0.010273176f0, "Oh lord!N")
 (0.0101707755f0, "Oh lord!q")
 (0.010157032f0, "Oh lord!E")

In [19]:
d=trainingData()
size(d)

(2178,)

In [38]:
CUDA.memory_status()

Effective GPU memory usage: 34.08% (5.019 GiB/14.726 GiB)
CUDA allocator usage: 4.837 GiB
binned usage: 4.837 GiB (4.837 GiB allocated, 0 bytes cached)


In [39]:
GC.gc(true)
CUDA.memory_status()

Effective GPU memory usage: 34.08% (5.019 GiB/14.726 GiB)
CUDA allocator usage: 4.837 GiB
binned usage: 4.837 GiB (48.856 MiB allocated, 4.789 GiB cached)


In [43]:
CUDA.memory_status()

using Statistics: mean
firsttime=true
avgsum=0 trainingData(lengthOfSentences,numberOfSentencesInBatch)
for next in 
   x=next
   if firsttime
    @show typeof(x), typeof(next)
    firsttime=false
    end
    avgsum+=mean(x[1])
    end
    @show avgsum
    
    
CUDA.memory_status()
GC.gc(true)
CUDA.memory_status()

Effective GPU memory usage: 34.08% (5.019 GiB/14.726 GiB)
CUDA allocator usage: 4.837 GiB
binned usage: 4.837 GiB (48.856 MiB allocated, 4.789 GiB cached)
(typeof(x), typeof(next)) = (Tuple{Array{Int64,2},Transformers.Basic.OneHotArray{0x00000042,1,2,Array{Transformers.Basic.OneHot{0x00000042},1}}}, Tuple{Array{Int64,2},Transformers.Basic.OneHotArray{0x00000042,1,2,Array{Transformers.Basic.OneHot{0x00000042},1}}})
avgsum = 84087.02460629922
Effective GPU memory usage: 34.08% (5.019 GiB/14.726 GiB)
CUDA allocator usage: 4.837 GiB
binned usage: 4.837 GiB (48.856 MiB allocated, 4.789 GiB cached)
Effective GPU memory usage: 34.08% (5.019 GiB/14.726 GiB)
CUDA allocator usage: 4.837 GiB
binned usage: 4.837 GiB (48.856 MiB allocated, 4.789 GiB cached)


In [51]:
sum(length(p)  for (p,x) in trainingData())

141622272

In [29]:
CUDA.allowscalar(true)

decay = 0.1
optWithWeightDecay=Optimiser(opt,
			Train.SelectiveWeightDecay(WeightDecay(decay), Model.noDecayList(model))) |> gpu


ps = params(model)

batchSize=64
  d=trainingData(lengthOfSentences,numberOfSentencesInBatch) 
  Train.train!(loss,
            ps,
            d,
            optWithWeightDecay,
            sampleFromModel, repeat=3 )




(ctr, numberOfBatches) = (0, (2178,))


LoadError: ignored

In [16]:
opt


ADAM(0.0005992063380936809, (0.9, 0.95), IdDict{Any,Any}(Float32[0.021266753, 0.023991061, -0.014033463, -0.020187732, -0.016970964, -0.023259092, 0.019853802, -0.011446846, 0.01508859, -0.012974098  …  -0.018857775, 0.017281272, 0.009872336, 0.018419951, -0.00944189, -0.018381367, 0.021812147, 0.025983006, -0.007428212, -0.0012844022] => (Float32[-3.2025746f-6, -2.5215654f-6, 1.0904602f-6, -1.7421075f-6, 2.7151017f-7, 1.4392883f-5, 5.462291f-7, 1.4074294f-6, -6.43307f-7, -1.0274645f-5  …  -4.5363145f-6, -8.378738f-6, -6.2535555f-6, 3.1016405f-6, -3.885244f-6, -1.2411709f-5, -1.1541694f-5, -1.5393276f-6, 3.0699089f-6, 8.032217f-7], Float32[2.922207f-9, 2.2120228f-9, 6.332865f-9, 4.241979f-9, 6.004763f-9, 8.711295f-9, 3.5520626f-9, 3.8897174f-9, 3.4757772f-9, 4.3774055f-9  …  8.143284f-9, 4.1621586f-9, 3.5048722f-9, 3.6281622f-9, 4.7192823f-9, 4.703792f-9, 5.3198157f-9, 5.072381f-9, 6.3835452f-9, 6.998285f-9], (1.613975805220401e-10, 1.7094308280841148e-5)),Float32[-0.0005461035 -0.0008

In [ ]:
model.pw.models[1].b


66-element CuArray{Float32,1}:
 -1.5513743
 -0.0863119
  0.16213056
 -0.2024484
 -0.92951417
 -0.5469302
 -0.034112763
 -0.0040509147
 -0.23159942
 -0.12889002
 -0.22430605
 -0.06451803
 -0.19396299
  ⋮
  0.023614747
 -0.052799366
 -0.19879629
  0.019089265
  0.16964178
  0.10373718
 -0.07236537
 -0.13919203
 -0.004549714
 -0.25447264
 -0.05559869
 -0.25983748

In [ ]:
Pkg.add("BSON")
using BSON

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [ ]:
using BSON: 
@save "inputtext_model.bson"  model

In [ ]:
using Zygote
using Flux
using Transformers
using CUDA
using BSON:  @save, @load
@load "inputtext_model.bson"  model


In [ ]:
sampleFromModel("Where is King Richard Duke of York, and ", 0.001)


LoadError: ignored

In [ ]:
Pkg.add("StatsBase")
Pkg.add("Plots")

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
  Installed Ogg_jll ───────── v1.3.4+2
  Installed LAME_jll ──────── v3.100.0+3
  Installed x265_jll ──────── v3.0.0+3
  Installed PlotThemes ────── v1.0.3
  Installed RecipesPipeline ─ v0.1.13
  Installed Plots ─────────── v1.0.14
  Installed Contour ───────── v0.5.7
  Installed GeometryTypes ─── v0.8.4
  Installed Showoff ───────── v0.3.2
  Installed Grisu ─────────── v1.0.0
  Installed FreeType2_jll ─── v2.10.1+5
  Installed Opus_jll ──────── v1.3.1+3
  Installed FFMPEG_jll ────── v4.3.1+4
  Installed LibVPX_jll ────── v1.9.0+1
  Installed libfdk_aac_jll ── v0.1.6+4
  Installed PlotUtils ─────── v0.6.5
  Installed libvorbis_jll ─── v1.3.6+6
  Installed FFMPEG ────────── v0.3.0
  Installed Bzip2_jll ─────── v1.0.6+5
  Installed Measures ──────── v0.3.1
  Installed FriBidi_jll ───── v1.0.5+6
  Installed libass_

In [ ]:
using StatsBase
using Plots
function getOutputResults(m, data, limit=5)
	output=[]
	ctr=0
	for (x,y) in data
		out=softmax(m(x))
		out=reshape(out,size(out)[1], :)
		push!(output, onecold(out)...)  ## get the encoded vocab using straight onecold, not onecold(vocab, ..) version

		ctr +=1
		if  ctr > limit break end
	end
	convert(Array{Int}, output)
end

d=trainingData(lengthOfSentences, numberOfSentencesInBatch)
function OutputHistogram(m)
	outChars=getOutputResults(m, d)
	fit(Histogram, outChars, 0:1:length(vocab))
end

#Try a untrained model
raw=OutputHistogram(Model.minGPT(maxBlockSize=lengthOfSentences, vocabSize=length(vocab)))

#..a. and trained model
trained=OutputHistogram(model)

plot(raw)
plot!(trained)

LoadError: ignored

In [ ]:
data = s 
collect(data)[1:10]

10-element Array{Char,1}:
 'F': ASCII/Unicode U+0046 (category Lu: Letter, uppercase)
 'i': ASCII/Unicode U+0069 (category Ll: Letter, lowercase)
 'r': ASCII/Unicode U+0072 (category Ll: Letter, lowercase)
 's': ASCII/Unicode U+0073 (category Ll: Letter, lowercase)
 't': ASCII/Unicode U+0074 (category Ll: Letter, lowercase)
 ' ': ASCII/Unicode U+0020 (category Zs: Separator, space)
 'C': ASCII/Unicode U+0043 (category Lu: Letter, uppercase)
 'i': ASCII/Unicode U+0069 (category Ll: Letter, lowercase)
 't': ASCII/Unicode U+0074 (category Ll: Letter, lowercase)
 'i': ASCII/Unicode U+0069 (category Ll: Letter, lowercase)

In [21]:
GC.gc(true)
CUDA.memory_status()


Effective GPU memory usage: 1.54% (231.875 MiB/14.726 GiB)
CUDA allocator usage: 48.856 MiB
binned usage: 48.856 MiB (48.856 MiB allocated, 0 bytes cached)


In [ ]:
model=nothing; GC.gc(true)

In [ ]:
CUDA.memory_status()


Effective GPU memory usage: 8.28% (1.219 GiB/14.726 GiB)
CUDA allocator usage: 1.047 GiB
binned usage: 1.047 GiB (0 bytes allocated, 1.047 GiB cached)


In [ ]:
CUDA.reclaim()

1124073472

In [ ]:
CUDA.memory_status()


Effective GPU memory usage: 1.17% (175.875 MiB/14.726 GiB)
CUDA allocator usage: 0 bytes
binned usage: 0 bytes (0 bytes allocated, 0 bytes cached)
